## Reinforcement Learning Sprint Challenge - play Taxi 
Solution - Jason Fleischer

###Getting Setup

In [2]:
# install packages not yet installed on colab machine

!pip install gym

In [0]:
# import needed packages

import gym
import numpy as np
import random
from collections import defaultdict

In [4]:
# create environment for 'Taxi-v2'

env = gym.make('Taxi-v2')

# see what it looks like

env.render()

+---------+
|R: | : :G|
| : : : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+



In [5]:
# random agent playing taxi (taken from Sprint Challenge instructions)

total_reward = 0
done = False
env.reset()
while not done:
    state, reward, done, info = env.step(env.action_space.sample())
    total_reward += reward
    env.render()

print('Total reward:', total_reward)

+---------+
|R: | : :G|
| : : : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (Pickup)
+---------+
|R: | : :G|
| : : : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (Dropoff)
+---------+
|R: | : :G|
| : : : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (South)
+---------+
|R: | : :G|
| : : : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (West)
+---------+
|R: | : :G|
| : : : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (West)
+---------+
|R: | : :G|
| : : : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (West)
+---------+
|R: | : :G|
| : : : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (North)
+---------+
|R: | : :G|
| : : : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (West)
+---------+
|R: | : :G|
| : : : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (Dropoff)
+---------+
|R: | : :G|
| : : : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (East)
+---------+
|R: | : :G|
| : : : : |
| : : : : |
| | : | : |


In [0]:
# find attribute for action and observation spaces
# output cleared for readability

dir(env.action_space)
dir(env.observation_space)

In [7]:
# find action and observation spaces

print('Action Size: ', env.action_space.n)
print('State Size: ', env.observation_space.n)

Action Size:  6
State Size:  500


In [0]:
# what do these spaces represent?
# help is not very illuminating
# output cleared for readability

help(gym.envs.toy_text.taxi)

In [0]:
# it appears from taxi.py, line 127 that the action space is 
# "South", "North", "East", "West", "Pickup", and "Dropoff"

In [10]:
# the observation space is encoded as an integer between 0 and 499
print(list(gym.envs.toy_text.taxi.TaxiEnv.decode(env, 0)))
print(list(gym.envs.toy_text.taxi.TaxiEnv.decode(env, 499)))

[0, 0, 0, 0]
[4, 4, 4, 3]


In [0]:
# it is clear from taxi.py that the first two indices refer to the row and col
# location of the taxi

# for i in range(500):
#     state = list(gym.envs.toy_text.taxi.TaxiEnv.decode(env, i))
#     print(state[2])
#     print(state[3])

# my guess for the third and fourth index:
# third index (passidx): refers to whether the passenger is at one of the 
# four locations or in the taxi
# fourth index (destidx): refers to which of the four locations the destination
# is at

# the map never changes


###Goal 1 - Beat Random

In [0]:
# create an empty qtable

qtable = np.zeros((500, 6))

In [0]:
# Basic q-learning hyperparameter setup

episodes = 100000
max_steps = 99

learning_rate = 0.6
gamma = 0.7

# Exploration parameters

epsilon = 1.0
max_epsilon = 1.0
min_epsilon = 0.01
decay_rate = 0.01

In [25]:
# use template from Sprint Challenge instructions to measure performance over
# a number of episodes

# start with empty list of rewards to be updated and later averaged
rewards = []

for episode in range(episodes):
    state = env.reset() # reset state for each episode
    total_rewards = 0 # start with 0 rewards for each episode
    
    for step in range(max_steps):

        # decide to explore or exploit
        
        if random.uniform(0, 1) < epsilon:
            
            # explore
            action = env.action_space.sample()
            
        else:
            
            # exploit
            action = np.argmax(qtable[state,:])
        
        # return state and reward after action is taken
        new_state, reward, done, info = env.step(action)
        
        # update qtable value based on Bellman equation
        qtable[state, action] = qtable[state, action] + learning_rate * (reward + gamma * np.max(qtable[new_state, :]) - qtable[state, action])
        
        # increment total_rewards for episode by reward for action
        total_rewards += reward
        
        # update state
        state = new_state
        
        # break loop if taxi drops off passenger
        if done:
            break
            
    # reduce epsilon to use current qtable value more over exploring
    epsilon = (min_epsilon +
               (max_epsilon - min_epsilon) * 
               np.exp(-decay_rate * episode))
    
    # when finished add total_rewards to list of rewards for all episodes
    rewards.append(total_rewards)             

print('Average score over time:', sum(rewards) / episodes)

Average score over time: 7.45056


###Goal 2 - Beat Basic Q-Learning

In [0]:
# create an empty qtable

qtable_1 = np.zeros((500, 6))

In [0]:
# tweak hyperparameters to achieve better score

episodes_1 = 1000000
max_steps_1 = 40

learning_rate_1 = 0.5
gamma_1 = 0.8

epsilon_1 = 1.0
max_epsilon_1 = 1.0
min_epsilon_1 = 0.001
decay_rate_1 = 0.05

In [44]:
rewards_1 = []

for episode_1 in range(episodes_1):
    state = env.reset()
    total_rewards_1 = 0
    
    for step in range(max_steps_1):
        
        if random.uniform(0, 1) < epsilon_1:
            action = env.action_space.sample()
            
        else:
            action = np.argmax(qtable_1[state,:])

        new_state, reward, done, info = env.step(action)
        
        qtable_1[state, action] = qtable_1[state, action] + learning_rate_1 * (reward + gamma_1 * np.max(qtable_1[new_state, :]) - qtable_1[state, action])
        
        total_rewards_1 += reward
        
        state = new_state
        
        if done:
            break
    
    epsilon_1 = (min_epsilon_1 +
               (max_epsilon_1 - min_epsilon_1) * np.exp(-decay_rate_1 * episode_1))
    
    rewards_1.append(total_rewards_1)             

print('Average score over time:', sum(rewards_1) / episodes_1)

Average score over time: 8.330848


In [0]:
# I should have put this in a function or object... oh well
# Trying again below to save the score above

In [0]:
# tweak hyperparameters to achieve better score

episodes_1 = 1000000
max_steps_1 = 50

learning_rate_1 = 0.7
gamma_1 = 0.75

epsilon_1 = 1.0
max_epsilon_1 = 1.0
min_epsilon_1 = 0.001
decay_rate_1 = 0.05

In [49]:
rewards_1 = []

for episode_1 in range(episodes_1):
    state = env.reset()
    total_rewards_1 = 0
    
    for step in range(max_steps_1):
        
        if random.uniform(0, 1) < epsilon_1:
            action = env.action_space.sample()
            
        else:
            action = np.argmax(qtable_1[state,:])

        new_state, reward, done, info = env.step(action)
        
        qtable_1[state, action] = qtable_1[state, action] + learning_rate_1 * (reward + gamma_1 * np.max(qtable_1[new_state, :]) - qtable_1[state, action])
        
        total_rewards_1 += reward
        
        state = new_state
        
        if done:
            break
    
    epsilon_1 = (min_epsilon_1 +
               (max_epsilon_1 - min_epsilon_1) * np.exp(-decay_rate_1 * episode_1))
    
    rewards_1.append(total_rewards_1)             

print('Average score over time:', sum(rewards_1) / episodes_1)

Average score over time: 8.369608


In [0]:
# So close...

###Goal 3 - Beat Optimized Q-Learning

My initial strategy was to break out the encoded state into the four pieces of information it encodes to see whether Q-Learning would be more effective.  After some thought, I think Q-Learning would be agnostic to whether the information was endoded or not.

The np.array needed was too large for Colab, but I could have possibly tried the defaultdict method and only created entries as needed.

In [0]:
# qtable_2 = np.zeros((500,500,500,500, 6))

In [0]:
rewards = []

for episode in range(total_episodes):
    state_encoded = env.reset()
    done = False
    total_rewards = 0
    
    for step in range(max_steps):
        state = list(gym.envs.toy_text.taxi.TaxiEnv.decode(env, state_encoded))
        taxi_row = state[0] 
        taxi_col = state[1]
        pass_idx = state[2]
        dest_idx = state[3]
        
        if random.uniform(0, 1) < epsilon:
            
            action = env.action_space.sample()
            
        else:
            
            action = np.argmax(qtable_2[taxi_row, 
                                      taxi_col, 
                                      pass_idx, 
                                      dest_idx , :])
        

        new_state, reward, done, info = env.step(action)
        
        qtable_2[taxi_row, 
                 taxi_col, 
                 pass_idx, 
                 dest_idx] = (qtable_2[taxi_row, 
                                       taxi_col, 
                                       pass_idx, 
                                       dest_idx] +
                              learning_rate *
                              (reward + gamma *
                               np.max(qtable_2[new_state[0], 
                                               new_state[1], 
                                               new_state[2],
                                               new_state[3]:]) -
                               qtable[taxi_row, taxi_col, pass_idx,dest_idx, action]))
        

        
        total_rewards += reward
        
        state = new_state
        
        if done:
            break
    
    epsilon = (min_epsilon +
              (max_epsilon - min_epsilon) * np.exp(-decay_rate * episode))
    rewards.append(total_rewards)
    
print('Score over time:', sum(rewards) / total_episodes)